<a href="https://kritikseth.github.io/ipynbtagredirect" target="_parent"><img src="https://raw.githack.com/kritikseth/kritikseth/master/assets/icons/kritik_ipynbtagredirect.svg" alt="Kritik Seth"/></a>

Purpose of this notebook is to build a CNN model and make a flask application to predict numbers all in one place.
To try out the application. Click on Copy and Edit button on top-right of the notebook, uncomment the last cell and press Run All

<h1 style="background-color:#2d6187;color:white;font-family:Arial;font-size:350%;text-align:center">CNN</h1>

### Importing Libraries

In [ ]:
import pandas as pd
import numpy as np

import os
import cv2
from PIL import Image

from tqdm.auto import trange, tqdm

import keras
import tensorflow as tf
from tqdm.keras import TqdmCallback

from matplotlib import pyplot as plt
%matplotlib inline
from ipywidgets import interact

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

In [ ]:
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test = np.genfromtxt('/kaggle/input/digit-recognizer/test.csv', delimiter=',', skip_header=1)
sample_submission = pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv')

In [ ]:
train.shape, test.shape, sample_submission.shape

In [ ]:
labels = train['label']
train.drop(['label'], axis=1, inplace=True)

In [ ]:
train = np.array(train, dtype=float)

Reshaping train and images test to 2D

In [ ]:
train = train.reshape(42000, 28, 28)
test = test.reshape(28000, 28, 28)

Resizing images from (28, 28) -> (32, 32)

In [ ]:
img_rows, img_cols = (32, 32)
input_shape = (img_rows, img_cols, 3)

In [ ]:
def resizeImage(images, new_size):
    nimages = np.zeros((images.shape[0], new_size[0], new_size[1], 3))
    for image in trange(images.shape[0]):
        nimages[image, :new_size[0], :new_size[1], 0] = cv2.resize(images[image], new_size)
        nimages[image, :new_size[0], :new_size[1], 1] = cv2.resize(images[image], new_size)
        nimages[image, :new_size[0], :new_size[1], 2] = cv2.resize(images[image], new_size)
    return nimages

In [ ]:
train = resizeImage(train, (img_rows, img_cols))
test = resizeImage(test, (img_rows, img_cols))

#### Augmenting Images

Image augmentation is one useful technique in building convolutional neural networks that can increase the size of the training set without acquiring new images. The idea is simple; duplicate images with some kind of variation so the model can learn from more examples.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
val_split = 0.05
BATCH_SIZE = 256
TRAIN_STEPS_PER_EPOCH = train.shape[0]*(1-val_split)//BATCH_SIZE
VAL_STEPS_PER_EPOCH = train.shape[0]*val_split//BATCH_SIZE

train_datagen = ImageDataGenerator(rescale=1/255.0,
                                   rotation_range=15,
                                   zoom_range=0.15,
                                   validation_split=val_split,
                                   width_shift_range=0.15,
                                   height_shift_range=0.15,
                                   shear_range=0.15,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1/255.0)

In [ ]:
train_aug = train_datagen.flow(train,
                               labels,
                               batch_size=BATCH_SIZE,
                               subset='training',
                               shuffle=False,
                               seed=42)

valid_aug = train_datagen.flow(train,
                               labels,
                               batch_size=BATCH_SIZE,
                               subset='validation',
                               shuffle=False,
                               seed=42)


test_aug = train_datagen.flow(test,
                              batch_size=BATCH_SIZE,
                              shuffle=False,
                              seed=42)

In [ ]:
def build_model():
    cnn = keras.models.Sequential()

    cnn.add(keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', kernel_initializer='random_uniform', input_shape=input_shape))
    cnn.add(keras.layers.Dropout(0.5))
    cnn.add(keras.layers.MaxPool2D(pool_size=(2, 2)))

    cnn.add(keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same', kernel_initializer='random_uniform'))
    cnn.add(keras.layers.Dropout(0.5))
    cnn.add(keras.layers.MaxPool2D(pool_size=(2, 2)))

    cnn.add(keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same', kernel_initializer='random_uniform'))
    cnn.add(keras.layers.Dropout(0.5))
    cnn.add(keras.layers.MaxPool2D(pool_size=(3, 3)))

    cnn.add(keras.layers.Flatten())
    cnn.add(keras.layers.Dense(units=128, activation='relu'))

    cnn.add(keras.layers.Dense(units=10, activation='softmax'))
    
    cnn.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['acc'])
    return cnn

In [ ]:
cnn = build_model()

In [ ]:
keras.utils.plot_model(cnn, show_shapes=True, show_layer_names=True)

**Callbacks**

Too many epochs can lead to overfitting of the training dataset, whereas too few may result in an underfit model. Callbacks provide a way to execute code and interact with the training model process automatically.

Here we implement the following callbacks:
* ModelCheckpoint- saves the model, when the accuracy is has the maximum value
* Early Stopping- stops the training of the model when the value of accuracy does not increase in a specified number of rounds by a specified value
* ReduceLRonPlateau- Decrease the value of learning rate by a factor when the accuracy is flattening out
* TqdmCallback- just a pretty way to keep track of epochs :)


In [ ]:
ch = tf.keras.callbacks.ModelCheckpoint(
    filepath='mnist_model.h5',
    save_weights_only=False,
    monitor='acc',
    mode='max',
    save_best_only=True
)

es = tf.keras.callbacks.EarlyStopping(
    monitor='acc',
    min_delta=0.003,
    patience=15,
    mode='max',
    restore_best_weights=True,
)

lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='acc',
    factor=0.05,
    patience=3,
    mode='max',
)

In [ ]:
cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [ ]:
history = cnn.fit(train_aug, validation_data=valid_aug, epochs=50, verbose=0,
                  callbacks=[ch, es, lr, TqdmCallback(verbose=1)])

In [ ]:
y_pred = cnn.predict_classes(test)

In [ ]:
sample_submission['Label'] = y_pred
sample_submission.to_csv('ss.csv', index=False)

In [ ]:
cnn.save('mnist_model.h5')

<h1 style="background-color:#2d6187;color:white;font-family:Arial;font-size:350%;text-align:center">Flask Application</h1>

This section of notebook focuses on building a flask application and running it using ngrok

In [ ]:
!pip install flask gevent requests pillow flask-ngrok -q

In [ ]:
import os
os.chdir('/kaggle/working')

Every web application needs an HTML file, this is ours. It's very simple, but gets the job done.
Feel free to replace this with yours!

In [ ]:
html = '''<!DOCTYPE HTML>
<!DOCTYPE HTML>
<html lang="en">
    <head>
        <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
        <title>MNIST Application</title>
        <link rel="stylesheet" href="https://codepen.io/chriddyp/pen/bWLwgP.css">
    </head>

    <body>

        <br>
        <center><h1 id="title">MNIST Flask Application</h1></center>
        <br>
        <form name="image_form" action="/" method="POST" enctype="multipart/form-data">
            <div style="width: 800px; margin: 0 auto; justify-content: center; align-items: center">

                <div style="width: 280px; height: 280px; float:left; border:1px solid rgb(100, 100, 100); font-size: 280px; line-height: 280px; text-align: center">
                    <img src="" id="preview" style="width: 280px; height: 280px; float:left;">
                </div>

                <div style="width: 230px; height: 280px; float: left; position:relative">
                    <input type="file" name="file" id="filetag" accept="image/png, image/jpeg, image/jpg" style="width: 75%; position:absolute; top: 30%; left: 12%">
                    <input id="predict" class="cust_button" type="submit" value="Upload" style="width: 75%; position:absolute; top: 50%; left: 12%"/> 
                </div>

                <div style="width: 280px; height: 280px; float:left; border:1px solid black; font-size: 280px; line-height: 280px; text-align: center">
                    <span id="num" style="">{{number}}</span>
                </div>
            </div>
        </form>
    </body>

    <script>
        var fileTag = document.getElementById("filetag"),
            preview = document.getElementById("preview");
            
        fileTag.addEventListener("change", function() {
        changeImage(this);
        });

        function changeImage(input) {
        var reader;

        if (input.files && input.files[0]) {
            reader = new FileReader();

            reader.onload = function(e) {
            preview.setAttribute('src', e.target.result);
            }

            reader.readAsDataURL(input.files[0]);
        }
        }
    </script>

</html>
'''

Saving the HTML file

In [ ]:
!mkdir templates
!mkdir uploads
HTML_file = open('templates/index.html', 'w')
HTML_file.write(html)
HTML_file.close()

#### Flask Application

The below code has been commented out in order to allow me to save this kernal. To run the web application, uncomment the below code and run it.

In [ ]:
# import os
# import cv2
# import keras
# from flask import Flask, render_template, request
# from flask_ngrok import run_with_ngrok
# import numpy as np
# from matplotlib import pyplot as plt

# app = Flask(__name__, static_folder='/kaggle/working/templates')
# run_with_ngrok(app)

# app.config['UPLOADS'] = 'uploads'

# cnn = keras.models.load_model('/kaggle/working/mnist_model.h5')

# def process(file):
#     image = cv2.imread(file)
#     image = cv2.resize(image, (32, 32))
#     image = np.resize(image, (1, 32, 32, 3))
#     image = image/255.0
#     image = 1-image
#     return image

# @app.route('/')
# def index():
#     return render_template('index.html')

# @app.route('/', methods=['GET', 'POST'])
# def predict():
#     if request.method == 'POST':
#         file = request.files['file']
#         filepath = f'uploads/{file.filename}'
#         file.save(filepath)
#         image = process(filepath)
#         print('process done')
#         prediction = cnn.predict_classes(image)
        
#         return render_template('index.html', number=prediction[0])

# if __name__ == '__main__':
#     app.run()

> P.S.: You can use this application in your phone browser as well, just copy paste the link and you are ready to go :)

Please Upvote the notebook if you like the notebook and appreciate the efforts! :D

Contact me:
* [LinkedIn](https://www.linkedin.com/in/kritikseth/)
* [GitHub](http://github.com/kritikseth)
* [Email](mailto:sethkritik@gmail.com)

<h4 style="background-color:#2d6187;color:white;font-family:Arial;font-size:350%;text-align:center">Thank you</h4>